## SVM, Adaboost, XGboost

### 載入套件

In [ ]:
# pip install xgboost

In [47]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from time import time
from sklearn import metrics
import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

### 讀檔

In [2]:
## cvaw + stock sign
filePath = "./preprocess/tokens+stockSign/"
df_cvaw_covid = pd.read_csv(filePath + "cvaw_covid_stockSign.csv")[["Valence_Sum","Arousal_Sum", "words_Num"]]
df_cvaw_stock = pd.read_csv(filePath + "cvaw_stock_stockSign.csv")[["Valence_Sum","Arousal_Sum", "words_Num"]]
df_stockSign_covid = pd.read_csv(filePath + "cvaw_covid_stockSign.csv")[["date", "stockRise_mask", "stockRise_testKits", "stockRise_vaccine"]]
df_stockSign_stock = pd.read_csv(filePath + "cvaw_stock_stockSign.csv")[["date", "stockRise_mask", "stockRise_testKits", "stockRise_vaccine"]]

### 函式：讀 tokens data 並併入情緒詞，切成 x_train, x_test

In [3]:
def SVD_embedding(tokens_name, df_cvaw):
    df_embedding = pd.read_csv("./TFIDF/" + tokens_name + "_stockSign_TFIDF_TruncatedSVD.csv")
    
    df_embedding_cvaw = pd.concat([df_embedding, df_cvaw], axis = 1).iloc[:, 1:504]
    
    x_train, x_test = train_test_split(df_embedding_cvaw, test_size=0.20, random_state=404)
    
    return x_train, x_test

In [4]:
def word2vec_embedding(tokens_name, df_cvaw):
    df_embedding = pd.read_csv("./Word2Vec/" + tokens_name + "_stockSign_word2vec.csv")
    
    df_embedding_cvaw = pd.concat([df_embedding, df_cvaw], axis = 1).iloc[:, 1:504]
    
    x_train, x_test = train_test_split(df_embedding_cvaw, test_size=0.20, random_state=404)
    
    return x_train, x_test

### 口罩類股 斷詞 testing & training split

In [5]:
x_train_mask, x_test_mask = word2vec_embedding("covid_token", df_cvaw_covid)
y_train_mask, y_test_mask = train_test_split(df_stockSign_covid[["stockRise_mask"]], test_size=0.20, random_state=404)

In [6]:
x_train_mask

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
7271,0.466977,0.490552,-0.046918,-0.202188,0.166707,0.052072,0.100822,0.127350,0.293276,0.144377,...,0.080226,-0.125111,-0.088842,0.191329,-0.090425,0.034194,-0.061873,-8.4,-5.6,8
317,-0.137972,0.238994,-0.103087,-0.276188,-0.096632,0.055066,-0.054771,0.103535,-0.027499,0.190077,...,-0.107904,-0.122517,-0.083494,0.085071,0.077354,0.072515,0.231514,-4.4,-13.2,28
1925,-0.272390,-0.219359,0.030242,0.378799,0.014876,-0.015882,-0.218432,0.099289,0.055401,0.247769,...,-0.022874,0.055691,-0.276878,0.057967,0.295714,-0.024754,0.151208,-0.6,-3.4,3
3188,-0.165721,0.238822,-0.049381,-0.021363,0.317870,-0.113444,-0.313132,0.128785,0.005647,0.069155,...,0.234333,-0.128627,0.060161,0.052233,-0.132020,0.074416,0.239536,11.8,-10.2,17
7289,-0.231800,-0.203382,0.166360,-0.131830,0.190765,-0.032649,0.228955,-0.392035,-0.171504,0.106874,...,-0.069726,-0.126730,0.170638,0.217938,0.494262,0.491883,0.135065,-17.8,-2.6,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,0.635976,0.370562,-0.194026,-0.273566,0.023036,0.088587,0.273419,0.192486,0.337552,0.173713,...,-0.355655,-0.181885,-0.530296,-0.050131,0.032169,-0.168377,-0.005768,-1.2,-40.0,56
5108,-0.282629,-0.084010,-0.158663,0.005067,-0.008301,-0.057799,-0.283823,-0.172922,-0.028213,0.138294,...,-0.076541,-0.203598,-0.086399,-0.062242,-0.176672,-0.209389,0.280768,-5.8,-3.0,54
5302,-0.183439,-0.147837,-0.012782,0.516806,-0.024616,-0.445040,0.228963,-0.249978,-0.056679,0.074657,...,-0.307480,-0.170445,-0.036629,-0.077072,0.211082,-0.150104,0.504225,8.6,-5.5,13
5994,0.219891,0.571944,0.013836,-0.294966,0.212134,0.060913,0.161956,-0.021638,0.250821,-0.284589,...,0.037604,0.034764,-0.154887,0.087008,-0.282653,-0.279951,-0.046932,-9.3,-5.1,12


In [7]:
y_train_mask

,stockRise_mask
7271,0
317,-1
1925,1
3188,1
7289,-1
...,...
6031,-1
5108,1
5302,0
5994,0


In [8]:
y_train_mask["stockRise_mask"].value_counts()

 1    2374
-1    2305
 0    1332
Name: stockRise_mask, dtype: int64

In [9]:
rise_mask_stock_index = (y_train_mask[["stockRise_mask"]] == 1).any(axis=1)
rise_mask_stock_df = y_train_mask[["stockRise_mask"]][rise_mask_stock_index]
rise_mask_stock_df

,stockRise_mask
1925,1
3188,1
581,1
1219,1
1264,1
...,...
3778,1
5437,1
4721,1
5108,1


In [10]:
rise_mask_stock_df_under = rise_mask_stock_df.sample(n = 1332, random_state=404)
rise_mask_stock_df_under

,stockRise_mask
4562,1
1599,1
3045,1
2924,1
6363,1
...,...
3032,1
6177,1
1385,1
2889,1


In [11]:
fall_mask_stock_index = (y_train_mask[["stockRise_mask"]] == -1).any(axis=1)
fall_mask_stock_df = y_train_mask[["stockRise_mask"]][fall_mask_stock_index]
fall_mask_stock_df

,stockRise_mask
317,-1
7289,-1
6756,-1
7480,-1
4612,-1
...,...
7470,-1
5205,-1
3545,-1
3579,-1


In [12]:
fall_mask_stock_df_under = fall_mask_stock_df.sample(n = 1332, random_state=404)
fall_mask_stock_df_under

,stockRise_mask
3581,-1
2026,-1
3571,-1
5486,-1
4301,-1
...,...
3910,-1
5875,-1
5956,-1
5201,-1


In [13]:
hold_mask_stock_index = (y_train_mask[["stockRise_mask"]] == 0).any(axis=1)
hold_mask_stock_df = y_train_mask[["stockRise_mask"]][hold_mask_stock_index]
hold_mask_stock_df

,stockRise_mask
7271,0
3695,0
5828,0
2135,0
713,0
...,...
3595,0
2119,0
2657,0
5302,0


In [14]:
y_train_mask_under = pd.concat([rise_mask_stock_df_under, fall_mask_stock_df_under, hold_mask_stock_df], axis = 0)
y_train_mask_under

,stockRise_mask
4562,1
1599,1
3045,1
2924,1
6363,1
...,...
3595,0
2119,0
2657,0
5302,0


In [15]:
y_train_mask_under.index

Int64Index([4562, 1599, 3045, 2924, 6363, 6477, 3181, 3290, 6880, 6560,
            ...
              63, 5386, 5509, 2423,   56, 3595, 2119, 2657, 5302, 5994],
           dtype='int64', length=3996)

In [16]:
x_train_mask_under = x_train_mask.loc[y_train_mask_under.index]
x_train_mask_under

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
4562,-0.103974,-0.162923,0.048376,0.275587,-0.055360,-0.191940,-0.022160,-0.238066,0.059698,0.125430,...,-0.076550,-0.076919,-0.118575,0.057378,0.141668,0.055998,0.230349,-0.9,-14.9,27
1599,-0.243404,-0.081229,0.134533,-0.166375,-0.196209,-0.123993,0.325815,-0.157416,0.012790,0.062287,...,-0.266711,0.011254,-0.041866,0.027328,0.571218,-0.022271,0.104431,0.6,-6.2,12
3045,0.017590,-0.015423,0.056822,0.036282,-0.069918,0.155769,0.169846,-0.216314,0.491080,0.033142,...,-0.228948,-0.093012,-0.123121,0.023322,0.276916,0.166701,-0.117918,-7.2,-7.0,9
2924,0.331805,0.417182,-0.187965,-0.334285,0.002425,0.035407,0.098045,0.192391,0.393140,0.159591,...,-0.272255,-0.236945,-0.056996,-0.135340,-0.085426,-0.088976,0.012532,6.7,-12.6,23
6363,-0.350903,-0.676611,-0.102414,0.262314,0.044192,-0.271391,-0.339529,0.390935,0.710168,0.180003,...,-0.176433,-0.146061,-0.043850,0.065162,0.113028,0.085920,-0.214765,-5.6,-1.4,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,-0.067584,0.012416,-0.159461,0.344823,-0.054138,-0.110580,0.010604,0.351025,0.202320,0.097251,...,0.059234,0.129470,-0.199978,-0.044688,0.147962,-0.028341,-0.057203,-8.3,-1.3,12
2119,-0.329718,-0.280909,-0.237084,0.052169,-0.176001,-0.186304,-0.346105,0.308577,0.555185,0.048337,...,-0.210226,-0.182584,-0.294932,0.167580,0.188871,0.263966,-0.177752,-12.5,6.8,15
2657,-0.396625,-0.286286,-0.113221,0.138397,-0.090793,-0.135257,-0.129395,-0.160155,-0.089059,0.148563,...,-0.101387,-0.015268,-0.385753,-0.085508,0.081875,-0.192013,0.268307,1.7,8.7,52
5302,-0.183439,-0.147837,-0.012782,0.516806,-0.024616,-0.445040,0.228963,-0.249978,-0.056679,0.074657,...,-0.307480,-0.170445,-0.036629,-0.077072,0.211082,-0.150104,0.504225,8.6,-5.5,13


In [17]:
print(x_train_mask_under.shape)
print(y_train_mask_under.shape)
print(x_test_mask.shape)
print(y_test_mask.shape)

(3996, 503)
(3996, 1)
(1503, 503)
(1503, 1)


In [18]:
x_train_mask_under = x_train_mask_under.values.tolist()
x_test_mask = x_test_mask.values.tolist()
y_train_mask_under = y_train_mask_under["stockRise_mask"].tolist()
y_test_mask = y_test_mask["stockRise_mask"].tolist()

### 檢測試劑類股 斷詞 testing & training split

In [19]:
x_train_testKits, x_test_testKits = SVD_embedding("stock_token", df_cvaw_stock)
y_train_testKits, y_test_testKits = train_test_split(df_stockSign_stock[["stockRise_testKits"]], test_size=0.20, random_state=404)

In [20]:
x_train_testKits

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
6004,0.110194,-0.056617,0.016188,0.051068,0.024915,-0.024274,-0.014723,-0.026258,-0.055791,-0.005208,...,0.003994,0.017639,-0.015786,0.002318,0.026095,0.000526,0.005658,3.0,-2.6,12
6702,0.215222,-0.029807,-0.106440,-0.060623,-0.081690,-0.008828,-0.005497,0.125946,-0.049947,-0.023866,...,-0.016545,-0.014984,0.025858,-0.018918,-0.001705,0.016183,0.027335,35.6,-22.2,48
4971,0.190443,0.456539,-0.089589,0.113550,0.085209,-0.011482,-0.013740,0.040625,-0.076117,0.018417,...,-0.018082,0.016909,0.012102,0.003540,-0.022592,0.000633,-0.007836,14.0,-12.4,29
1867,0.230840,-0.021076,-0.092870,-0.061819,-0.110322,-0.008187,0.002349,0.030462,0.027457,0.002368,...,0.010719,-0.002002,0.002424,0.010414,-0.025819,0.006383,0.013708,8.2,-33.6,39
2484,0.153676,-0.052038,0.060150,0.017483,0.192235,0.037284,-0.009671,0.023961,0.167582,-0.376188,...,0.012238,-0.005709,0.024119,-0.035278,0.012393,-0.016911,0.011092,0.1,-15.9,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,0.114674,0.005346,-0.026531,-0.063207,0.020567,0.034371,0.034781,-0.019504,-0.047736,0.029909,...,-0.007536,0.015456,0.012212,0.022440,0.019762,-0.006434,-0.001645,-5.5,6.6,18
5108,0.140593,-0.014271,-0.065399,-0.031681,-0.076528,-0.009261,0.001531,0.053130,-0.032995,-0.036033,...,0.045533,0.011390,0.017935,0.011710,0.002624,-0.019156,-0.023624,50.5,-27.6,57
5302,0.080773,-0.011286,0.018688,0.016437,-0.017477,-0.009216,-0.000077,-0.032430,0.000492,0.047174,...,0.004887,-0.012008,0.003088,0.024134,0.012363,-0.005537,0.018533,3.6,-5.8,13
5884,0.158278,0.128662,-0.078809,0.024062,0.000665,-0.018054,0.009116,0.010543,0.001331,-0.067082,...,0.008776,-0.010579,0.002256,-0.008569,0.012491,-0.017333,-0.002944,12.6,-25.0,39


In [21]:
y_train_testKits

,stockRise_testKits
6004,0
6702,1
4971,-1
1867,-1
2484,-1
...,...
6031,0
5108,-1
5302,0
5884,1


In [22]:
y_train_testKits["stockRise_testKits"].value_counts()

 0    3277
 1    2310
-1    2196
Name: stockRise_testKits, dtype: int64

In [23]:
rise_testKits_stock_index = (y_train_testKits[["stockRise_testKits"]] == 1).any(axis=1)
rise_testKits_stock_df = y_train_testKits[["stockRise_testKits"]][rise_testKits_stock_index]
rise_testKits_stock_df

,stockRise_testKits
6702,1
4089,1
7762,1
2458,1
9504,1
...,...
5205,1
3595,1
4721,1
5884,1


In [24]:
rise_testKits_stock_df_under = rise_testKits_stock_df.sample(n = 2196, random_state=404)
rise_testKits_stock_df_under

,stockRise_testKits
1953,1
5550,1
7134,1
4230,1
8427,1
...,...
4794,1
7118,1
7798,1
2241,1


In [25]:
fall_testKits_stock_index = (y_train_testKits[["stockRise_testKits"]] == -1).any(axis=1)
fall_testKits_stock_df = y_train_testKits[["stockRise_testKits"]][fall_testKits_stock_index]
fall_testKits_stock_df

,stockRise_testKits
4971,-1
1867,-1
2484,-1
4026,-1
1099,-1
...,...
4414,-1
7267,-1
5509,-1
3778,-1


In [26]:
hold_testKits_stock_index = (y_train_testKits[["stockRise_testKits"]] == 0).any(axis=1)
hold_testKits_stock_df = y_train_testKits[["stockRise_testKits"]][hold_testKits_stock_index]
hold_testKits_stock_df

,stockRise_testKits
6004,0
3197,0
4779,0
7331,0
7151,0
...,...
7359,0
2119,0
2657,0
6031,0


In [27]:
hold_testKits_stock_df_under = hold_testKits_stock_df.sample(n = 2196, random_state=404)
hold_testKits_stock_df_under

,stockRise_testKits
5727,0
8140,0
5815,0
8763,0
8765,0
...,...
2165,0
2059,0
8034,0
7530,0


In [28]:
y_train_testKits_under = pd.concat([rise_testKits_stock_df_under, fall_testKits_stock_df, hold_testKits_stock_df_under], axis = 0)
y_train_testKits_under

,stockRise_testKits
1953,1
5550,1
7134,1
4230,1
8427,1
...,...
2165,0
2059,0
8034,0
7530,0


In [29]:
y_train_testKits_under.index

Int64Index([1953, 5550, 7134, 4230, 8427, 7889,  112,  430, 5697, 5920,
            ...
            9462,  627, 2121, 1402, 6061, 2165, 2059, 8034, 7530,  686],
           dtype='int64', length=6588)

In [30]:
x_train_testKits_under = x_train_testKits.loc[y_train_testKits_under.index]
x_train_testKits_under

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
1953,0.068011,0.013826,0.034591,-0.019046,0.000292,0.061531,0.010391,-0.002927,0.006710,0.023376,...,-0.008872,-0.003456,-0.016838,-0.028471,0.030829,0.012924,-0.036627,1.4,-0.6,1
5550,0.079063,-0.034280,-0.049850,0.037088,0.012954,-0.006524,-0.002633,-0.058755,-0.019318,-0.007313,...,-0.017670,0.012541,-0.007741,-0.001951,0.004041,-0.004589,0.008070,10.4,-4.5,24
7134,0.153176,0.002502,-0.023902,-0.053709,-0.025227,0.018068,0.028229,-0.086281,0.064948,-0.022676,...,0.003611,0.038224,0.006548,-0.015166,0.023351,0.000113,0.007214,-23.2,3.1,45
4230,0.139174,0.016702,-0.034842,-0.101984,-0.000597,0.049364,0.018304,-0.033119,-0.025185,0.009301,...,-0.022070,-0.009142,-0.000856,-0.004973,-0.009974,-0.000510,0.013685,-14.7,1.6,43
8427,0.100959,-0.030376,0.000717,-0.002443,0.078472,-0.020115,0.048887,0.000551,0.023621,-0.047324,...,0.019515,-0.006633,0.007451,0.001187,-0.012062,0.021938,-0.008737,1.8,2.0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,0.164158,-0.033644,-0.066727,-0.051302,0.089052,0.028641,-0.014874,0.024573,0.009850,0.028962,...,0.001822,-0.005424,-0.013549,0.032463,0.044268,0.023244,-0.013924,-2.2,15.3,42
2059,0.146059,-0.036988,-0.051608,-0.001632,0.021313,-0.029141,0.044124,-0.060419,-0.013885,-0.072169,...,-0.019804,-0.001503,0.016325,0.052767,-0.020296,-0.062912,0.034942,9.0,-1.8,21
8034,0.223764,-0.061242,0.194062,0.078316,-0.054313,-0.106835,-0.004957,0.010000,-0.076754,-0.031107,...,-0.011021,-0.002137,0.002064,0.004312,0.024735,0.040875,-0.022438,12.9,-11.2,37
7530,0.220772,-0.004303,-0.077023,-0.140266,-0.016639,0.070333,0.013601,-0.070384,-0.013347,0.039231,...,-0.004476,-0.002164,0.020850,-0.019821,0.027594,0.014493,0.008990,-4.5,1.2,22


In [31]:
x_train_testKits_under = x_train_testKits_under.values.tolist()
x_test_testKits = x_test_testKits.values.tolist()
y_train_testKits_under = y_train_testKits_under["stockRise_testKits"].tolist()
y_test_testKits = y_test_testKits["stockRise_testKits"].tolist()

### 疫苗類股 斷詞 testing & training split

In [32]:
x_train_vaccine, x_test_vaccine = SVD_embedding("stock_token", df_cvaw_stock)
y_train_vaccine, y_test_vaccine = train_test_split(df_stockSign_stock[["stockRise_vaccine"]], test_size=0.20, random_state=404)

In [33]:
x_train_vaccine

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
6004,0.110194,-0.056617,0.016188,0.051068,0.024915,-0.024274,-0.014723,-0.026258,-0.055791,-0.005208,...,0.003994,0.017639,-0.015786,0.002318,0.026095,0.000526,0.005658,3.0,-2.6,12
6702,0.215222,-0.029807,-0.106440,-0.060623,-0.081690,-0.008828,-0.005497,0.125946,-0.049947,-0.023866,...,-0.016545,-0.014984,0.025858,-0.018918,-0.001705,0.016183,0.027335,35.6,-22.2,48
4971,0.190443,0.456539,-0.089589,0.113550,0.085209,-0.011482,-0.013740,0.040625,-0.076117,0.018417,...,-0.018082,0.016909,0.012102,0.003540,-0.022592,0.000633,-0.007836,14.0,-12.4,29
1867,0.230840,-0.021076,-0.092870,-0.061819,-0.110322,-0.008187,0.002349,0.030462,0.027457,0.002368,...,0.010719,-0.002002,0.002424,0.010414,-0.025819,0.006383,0.013708,8.2,-33.6,39
2484,0.153676,-0.052038,0.060150,0.017483,0.192235,0.037284,-0.009671,0.023961,0.167582,-0.376188,...,0.012238,-0.005709,0.024119,-0.035278,0.012393,-0.016911,0.011092,0.1,-15.9,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,0.114674,0.005346,-0.026531,-0.063207,0.020567,0.034371,0.034781,-0.019504,-0.047736,0.029909,...,-0.007536,0.015456,0.012212,0.022440,0.019762,-0.006434,-0.001645,-5.5,6.6,18
5108,0.140593,-0.014271,-0.065399,-0.031681,-0.076528,-0.009261,0.001531,0.053130,-0.032995,-0.036033,...,0.045533,0.011390,0.017935,0.011710,0.002624,-0.019156,-0.023624,50.5,-27.6,57
5302,0.080773,-0.011286,0.018688,0.016437,-0.017477,-0.009216,-0.000077,-0.032430,0.000492,0.047174,...,0.004887,-0.012008,0.003088,0.024134,0.012363,-0.005537,0.018533,3.6,-5.8,13
5884,0.158278,0.128662,-0.078809,0.024062,0.000665,-0.018054,0.009116,0.010543,0.001331,-0.067082,...,0.008776,-0.010579,0.002256,-0.008569,0.012491,-0.017333,-0.002944,12.6,-25.0,39


In [34]:
y_train_vaccine

,stockRise_vaccine
6004,0
6702,-1
4971,0
1867,0
2484,0
...,...
6031,0
5108,0
5302,1
5884,0


In [35]:
y_train_vaccine["stockRise_vaccine"].value_counts()

 0    4238
-1    1810
 1    1735
Name: stockRise_vaccine, dtype: int64

In [36]:
rise_vaccine_stock_index = (y_train_vaccine[["stockRise_vaccine"]] == 1).any(axis=1)
rise_vaccine_stock_df = y_train_vaccine[["stockRise_vaccine"]][rise_vaccine_stock_index]
rise_vaccine_stock_df

,stockRise_vaccine
4089,1
9504,1
3854,1
1189,1
7666,1
...,...
8248,1
3595,1
3778,1
5302,1


In [37]:
fall_vaccine_stock_index = (y_train_vaccine[["stockRise_vaccine"]] == -1).any(axis=1)
fall_vaccine_stock_df = y_train_vaccine[["stockRise_vaccine"]][fall_vaccine_stock_index]
fall_vaccine_stock_df

,stockRise_vaccine
6702,-1
1099,-1
2458,-1
1147,-1
8191,-1
...,...
8890,-1
81,-1
6882,-1
5509,-1


In [38]:
fall_vaccine_stock_df_under = fall_vaccine_stock_df.sample(n = 1735, random_state=404)
fall_vaccine_stock_df_under

,stockRise_vaccine
8024,-1
7022,-1
8775,-1
1138,-1
7686,-1
...,...
8071,-1
7411,-1
4522,-1
9714,-1


In [39]:
hold_vaccine_stock_index = (y_train_vaccine[["stockRise_vaccine"]] == 0).any(axis=1)
hold_vaccine_stock_df = y_train_vaccine[["stockRise_vaccine"]][hold_vaccine_stock_index]
hold_vaccine_stock_df

,stockRise_vaccine
6004,0
4971,0
1867,0
2484,0
4026,0
...,...
2119,0
4721,0
6031,0
5108,0


In [40]:
hold_vaccine_stock_df_under = hold_vaccine_stock_df.sample(n = 1735, random_state=404)
hold_vaccine_stock_df_under

,stockRise_vaccine
3736,0
6352,0
5627,0
6276,0
2303,0
...,...
4134,0
8301,0
7112,0
6143,0


In [41]:
y_train_vaccine_under = pd.concat([rise_vaccine_stock_df, fall_vaccine_stock_df_under, hold_vaccine_stock_df_under], axis = 0)
y_train_vaccine_under

,stockRise_vaccine
4089,1
9504,1
3854,1
1189,1
7666,1
...,...
4134,0
8301,0
7112,0
6143,0


In [42]:
y_train_vaccine_under.index

Int64Index([4089, 9504, 3854, 1189, 7666, 1203, 9161, 6741, 2888, 8247,
            ...
            5156, 4156, 3709, 5379,   33, 4134, 8301, 7112, 6143,  385],
           dtype='int64', length=5205)

In [43]:
x_train_vaccine_under = x_train_vaccine.loc[y_train_vaccine_under.index]
x_train_vaccine_under

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,Valence_Sum,Arousal_Sum,words_Num
4089,0.134586,0.013053,-0.021775,-0.079978,-0.041018,0.023323,0.016799,-0.046902,0.042857,-0.013254,...,-0.022548,0.012339,-0.002732,0.002095,0.029987,0.037744,0.032409,-25.9,-5.1,56
9504,0.085303,0.011604,-0.015621,-0.024435,-0.039070,0.002573,0.020788,-0.037175,0.138743,0.004479,...,0.023244,-0.013983,-0.016079,-0.019426,0.012024,0.001612,0.013378,5.8,-13.0,12
3854,0.114373,-0.004330,0.077317,0.037607,-0.041854,-0.037251,-0.007370,-0.027094,0.020505,0.002003,...,-0.012410,-0.016075,-0.011604,0.018776,0.014717,-0.003549,0.012633,0.2,-6.4,5
1189,0.138547,0.099062,0.015783,-0.093881,-0.002382,0.040915,0.017030,-0.047237,0.085916,0.037599,...,0.005451,-0.030987,-0.012924,0.021530,-0.019321,-0.016428,-0.001897,8.6,-18.4,25
7666,0.151387,0.018831,-0.027943,-0.065725,-0.031075,0.000898,0.003987,-0.027040,0.004095,0.080979,...,-0.002739,0.030739,0.020536,0.016505,-0.004611,0.005928,-0.003380,5.7,-11.4,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4134,0.213317,0.031932,0.005966,-0.176187,0.016287,0.076831,0.007535,-0.028373,-0.076244,0.061178,...,-0.005941,0.008579,-0.017073,-0.020394,0.010315,0.007977,-0.008951,-18.6,-23.9,51
8301,0.032147,-0.020166,0.027497,0.035180,0.075473,-0.139743,0.885105,0.065661,-0.057469,0.029344,...,0.001004,-0.004679,0.002134,-0.008065,-0.001320,-0.008657,0.003312,0.4,-2.8,4
7112,0.152626,0.196024,-0.055035,0.014645,-0.013129,-0.001584,0.001532,0.013545,-0.011311,0.014013,...,-0.026360,-0.008091,0.010215,-0.000568,0.021425,0.004631,0.002630,-4.3,-25.1,27
6143,0.092638,-0.031088,0.061028,0.017255,0.014833,-0.027329,0.010316,-0.001414,-0.001582,-0.019563,...,-0.006914,0.012894,0.032608,-0.015141,-0.003833,0.014621,0.002688,-2.6,2.4,3


In [44]:
x_train_vaccine_under = x_train_vaccine_under.values.tolist()
x_test_vaccine = x_test_vaccine.values.tolist()
y_train_vaccine_under = y_train_vaccine_under["stockRise_vaccine"].tolist()
y_test_vaccine = y_test_vaccine["stockRise_vaccine"].tolist()

## SVM Model 函式

In [47]:
def tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, Linear, embedding):
    
    if Linear == True:
        SVM_model = SVC(kernel = "linear", C = 1.0, probability=True)
    else:
        SVM_model = SVC(kernel = "rbf", gamma = "scale", C = 1.0, probability=True)
    
    t0 = time()
    SVM_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(SVM_model.predict(x_test))
    
    if Linear == True:
        print(tokens_name + " predict " + predict_stocks_name + "(SVM Linear + " + embedding + ")")
    else:
        print(tokens_name + " predict " + predict_stocks_name + "(SVM RBF + " + embedding + ")")
        
    print(metrics.classification_report(excepted_results,predicted_results))

## adaboost model 函式

In [134]:
def adaboost_tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, embedding, n_estimator):
    
    adaboost_model = AdaBoostClassifier(n_estimators = n_estimator, random_state = 404)
    
    t0 = time()
    adaboost_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(adaboost_model.predict(x_test))

    print(tokens_name + " predict " + predict_stocks_name + "(adaboost + " + embedding + ")")
        
    print(metrics.classification_report(excepted_results,predicted_results))

## xgboost model 函式（train, test 不能轉為 list）

In [249]:
def xgboost_tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, embedding, stock_type):
    
    y_test = y_test["stockRise_" + stock_type].tolist()
    
    xgboost_model = XGBClassifier(random_state = 404)
    
    t0 = time()
    xgboost_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(xgboost_model.predict(x_test))

    print(tokens_name + " predict " + predict_stocks_name + "(xgboost + " + embedding + ")")
        
    print(metrics.classification_report(excepted_results,predicted_results))

## Random forest 函式

In [45]:
def randomForest_tokens_predict_stocks(x_train, y_train, x_test, y_test, tokens_name, predict_stocks_name, embedding):
    
    randomForest_model = RandomForestClassifier(n_estimators=500, max_features=50, random_state=404)
    
    t0 = time()
    randomForest_model.fit(x_train, y_train)
    print("done in %0.3fs." % (time() - t0))
    
    predicted_results = []
    excepted_results = []

    excepted_results.extend(y_test)
    predicted_results.extend(randomForest_model.predict(x_test))

    print(tokens_name + " predict " + predict_stocks_name + "(random forest + " + embedding + ")")
        
    print(metrics.classification_report(excepted_results,predicted_results))

### 預測口罩類股漲跌效果（word2vec covid 板 ckip 斷詞）

In [48]:
tokens_predict_stocks(x_train_mask_under, y_train_mask_under, x_test_mask, y_test_mask, "undersampling Covid CKIP token", "mask stock", False, "word2vec")

done in 92.758s.
undersampling Covid CKIP token predict mask stock(SVM RBF + word2vec)
              precision    recall  f1-score   support

          -1       0.37      0.26      0.31       557
           0       0.23      0.58      0.33       324
           1       0.41      0.18      0.25       622

    accuracy                           0.30      1503
   macro avg       0.33      0.34      0.30      1503
weighted avg       0.35      0.30      0.29      1503



In [49]:
tokens_predict_stocks(x_train_mask_under, y_train_mask_under, x_test_mask, y_test_mask, "undersampling Covid CKIP token", "mask stock", True, "word2vec")

done in 321.803s.
undersampling Covid CKIP token predict mask stock(SVM Linear + word2vec)
              precision    recall  f1-score   support

          -1       0.38      0.36      0.37       557
           0       0.22      0.35      0.27       324
           1       0.44      0.33      0.38       622

    accuracy                           0.35      1503
   macro avg       0.35      0.35      0.34      1503
weighted avg       0.37      0.35      0.35      1503



In [135]:
adaboost_tokens_predict_stocks(x_train_mask_under, y_train_mask_under, x_test_mask, y_test_mask, "undersampling Covid CKIP token", "mask stock", "word2vec", 100)

done in 29.191s.
undersampling Covid CKIP token predict mask stock(adaboost + word2vec)
              precision    recall  f1-score   support

          -1       0.39      0.40      0.40       557
           0       0.26      0.36      0.30       324
           1       0.43      0.33      0.38       622

    accuracy                           0.36      1503
   macro avg       0.36      0.37      0.36      1503
weighted avg       0.38      0.36      0.37      1503



In [136]:
adaboost_tokens_predict_stocks(x_train_mask_under, y_train_mask_under, x_test_mask, y_test_mask, "undersampling Covid CKIP token", "mask stock", "word2vec", 500)

done in 152.470s.
undersampling Covid CKIP token predict mask stock(adaboost + word2vec)
              precision    recall  f1-score   support

          -1       0.41      0.41      0.41       557
           0       0.25      0.35      0.29       324
           1       0.41      0.32      0.36       622

    accuracy                           0.36      1503
   macro avg       0.36      0.36      0.35      1503
weighted avg       0.38      0.36      0.36      1503



In [250]:
xgboost_tokens_predict_stocks(x_train_mask_under, y_train_mask_under, x_test_mask, y_test_mask, "undersampling Covid CKIP token", "mask stock", "word2vec", "mask")

C:\Users\molly\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\molly\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[03:18:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
done in 52.660s.
undersampling Covid CKIP token predict mask stock(xgboost + word2vec)
              precision    recall  f1-score   support

          -1       0.41      0.38      0.40       557
           0       0.25      0.39      0.30       324
           1       0.44      0.33      0.38       622

    accuracy                           0.36      1503
   macro avg       0.36      0.37      0.36      1503
weighted avg       0.39      0.36      0.37      1503



In [48]:
randomForest_tokens_predict_stocks(x_train_mask_under, y_train_mask_under, x_test_mask, y_test_mask, "undersampling Covid CKIP token", "mask stock", "word2vec")

done in 68.911s.
undersampling Covid CKIP token predict mask stock(random forest + word2vec)
              precision    recall  f1-score   support

          -1       0.39      0.37      0.38       557
           0       0.26      0.41      0.32       324
           1       0.46      0.34      0.39       622

    accuracy                           0.37      1503
   macro avg       0.37      0.38      0.36      1503
weighted avg       0.39      0.37      0.37      1503



### 預測檢測試劑類股漲跌效果（SVD stock 板 jieba 斷詞）

In [69]:
tokens_predict_stocks(x_train_testKits_under, y_train_testKits_under, x_test_testKits, y_test_testKits, "undersampling Stock jieba token", "testKits stock",  False, "SVD vector")

done in 256.205s.
undersampling Stock jieba token predict testKits stock(SVM RBF + SVD vector)
              precision    recall  f1-score   support

          -1       0.20      0.03      0.05       525
           0       0.45      0.56      0.50       855
           1       0.28      0.40      0.33       566

    accuracy                           0.37      1946
   macro avg       0.31      0.33      0.29      1946
weighted avg       0.33      0.37      0.33      1946



In [70]:
tokens_predict_stocks(x_train_testKits_under, y_train_testKits_under, x_test_testKits, y_test_testKits, "undersampling Stock jieba token", "testKits stock",  True, "SVD vector")

done in 452.394s.
undersampling Stock jieba token predict testKits stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.30      0.36      0.33       525
           0       0.49      0.39      0.44       855
           1       0.32      0.37      0.34       566

    accuracy                           0.38      1946
   macro avg       0.37      0.37      0.37      1946
weighted avg       0.39      0.38      0.38      1946



In [181]:
adaboost_tokens_predict_stocks(x_train_testKits_under, y_train_testKits_under, x_test_testKits, y_test_testKits, "undersampling Stock jieba token", "testKits stock", "SVD vector", 100)

done in 45.183s.
undersampling Stock jieba token predict testKits stock(adaboost + SVD vector)
              precision    recall  f1-score   support

          -1       0.25      0.30      0.27       525
           0       0.46      0.38      0.41       855
           1       0.31      0.35      0.33       566

    accuracy                           0.35      1946
   macro avg       0.34      0.34      0.34      1946
weighted avg       0.36      0.35      0.35      1946



In [251]:
xgboost_tokens_predict_stocks(x_train_testKits_under, y_train_testKits_under, x_test_testKits, y_test_testKits, "undersampling Stock jieba token", "testKits stock", "SVD vector", "testKits")

[03:19:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
done in 104.561s.
undersampling Stock jieba token predict testKits stock(xgboost + SVD vector)
              precision    recall  f1-score   support

          -1       0.30      0.38      0.34       525
           0       0.50      0.37      0.43       855
           1       0.29      0.33      0.31       566

    accuracy                           0.36      1946
   macro avg       0.36      0.36      0.36      1946
weighted avg       0.39      0.36      0.37      1946



In [49]:
randomForest_tokens_predict_stocks(x_train_testKits_under, y_train_testKits_under, x_test_testKits, y_test_testKits, "undersampling Stock jieba token", "testKits stock", "SVD vector")

done in 124.907s.
undersampling Stock jieba token predict testKits stock(random forest + SVD vector)
              precision    recall  f1-score   support

          -1       0.32      0.40      0.36       525
           0       0.47      0.38      0.42       855
           1       0.34      0.36      0.35       566

    accuracy                           0.38      1946
   macro avg       0.38      0.38      0.38      1946
weighted avg       0.39      0.38      0.38      1946



### 預測疫苗類股漲跌效果（SVD stock 板 jieba 斷詞）

In [87]:
tokens_predict_stocks(x_train_vaccine_under, y_train_vaccine_under, x_test_vaccine, y_test_vaccine, "undersampling Stock jieba token", "vaccine stock",  False, "SVD vector")

done in 156.501s.
undersampling Stock jieba token predict vaccine stock(SVM RBF + SVD vector)
              precision    recall  f1-score   support

          -1       0.24      0.48      0.32       428
           0       0.62      0.20      0.30      1082
           1       0.22      0.38      0.28       436

    accuracy                           0.30      1946
   macro avg       0.36      0.35      0.30      1946
weighted avg       0.45      0.30      0.30      1946



In [88]:
tokens_predict_stocks(x_train_vaccine_under, y_train_vaccine_under, x_test_vaccine, y_test_vaccine, "undersampling Stock jieba token", "vaccine stock",  True, "SVD vector")

done in 329.743s.
undersampling Stock jieba token predict vaccine stock(SVM Linear + SVD vector)
              precision    recall  f1-score   support

          -1       0.26      0.35      0.30       428
           0       0.63      0.47      0.54      1082
           1       0.27      0.34      0.30       436

    accuracy                           0.42      1946
   macro avg       0.39      0.39      0.38      1946
weighted avg       0.47      0.42      0.43      1946



In [195]:
adaboost_tokens_predict_stocks(x_train_vaccine_under, y_train_vaccine_under, x_test_vaccine, y_test_vaccine, "undersampling Stock jieba token", "vaccine stock", "SVD vector", 100)

done in 36.367s.
undersampling Stock jieba token predict vaccine stock(adaboost + SVD vector)
              precision    recall  f1-score   support

          -1       0.24      0.35      0.28       428
           0       0.61      0.40      0.48      1082
           1       0.23      0.33      0.27       436

    accuracy                           0.37      1946
   macro avg       0.36      0.36      0.35      1946
weighted avg       0.44      0.37      0.39      1946



In [252]:
xgboost_tokens_predict_stocks(x_train_vaccine_under, y_train_vaccine_under, x_test_vaccine, y_test_vaccine, "undersampling Stock jieba token", "vaccine stock", "SVD vector", "vaccine")

[03:20:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
done in 147.269s.
undersampling Stock jieba token predict vaccine stock(xgboost + SVD vector)
              precision    recall  f1-score   support

          -1       0.25      0.37      0.30       428
           0       0.62      0.39      0.48      1082
           1       0.25      0.36      0.30       436

    accuracy                           0.38      1946
   macro avg       0.37      0.38      0.36      1946
weighted avg       0.46      0.38      0.40      1946



In [50]:
randomForest_tokens_predict_stocks(x_train_vaccine_under, y_train_vaccine_under, x_test_vaccine, y_test_vaccine, "undersampling Stock jieba token", "vaccine stock", "SVD vector")

done in 90.819s.
undersampling Stock jieba token predict vaccine stock(random forest + SVD vector)
              precision    recall  f1-score   support

          -1       0.27      0.41      0.32       428
           0       0.62      0.42      0.50      1082
           1       0.26      0.34      0.29       436

    accuracy                           0.40      1946
   macro avg       0.38      0.39      0.37      1946
weighted avg       0.46      0.40      0.41      1946

